## Robust Query Client
This notebook is all about using the SpotifyAPI class to obtain an access token and making more complex search query requests to Spotify.

In [1]:
import requests
import base64
import datetime
from urllib.parse import urlencode

In [2]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"

    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
        self.perform_auth()
        
    def get_client_credentials(self):
        """
        Returns a base64 encoded String.
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_secret == None or client_id == None:
            raise Exception("client_id and client_secret required.")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return { 
        "Authorization" : f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
        "grant_type" : "client_credentials"
        }
        
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200,299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_headers(self):
        access_token = self.get_access_token()
        headers = {"Authorization": f"Bearer {access_token}"}
        return headers
    
    def get_resource(self, lookup_id, resource_type="albums", version="v1"):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_headers()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_artist(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="artists")
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, resource_type="albums")
    
    def base_search(self, query_params):
        lookup_url = f"https://api.spotify.com/v1/search?{query_params}"
        headers = self.get_resource_headers()
        
        print(lookup_url)
        r = requests.get(lookup_url, headers=headers)
        print(r.status_code)
        if not r.status_code in range(200, 299):
            return {}
        return r.json()
    
    def search(self, query=None, search_type="artist", market_type="GB"):
        new_query=""
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            for key,value in query.items():
                new_query += f"{key}:{value} "
        query_params = urlencode({"q" : new_query, "type" : search_type.lower(), "market" : market_type})
        return self.base_search(query_params)
        
        

### SEARCH
Let's try searching for something!

In [3]:
client_id = "a3aadf7f4d184af0af9d849d96e89b12"
client_secret = ""

In [4]:
spotify = SpotifyAPI(client_id, client_secret)
tracks = spotify.search({"track": "another"}, search_type="track")

https://api.spotify.com/v1/search?q=track%3Aanother+&type=track&market=GB
200


In [60]:
for track in tracks:
    for i in range(len(tracks['tracks']['items'])):
        print("Track id: " + tracks['tracks']['items'][i]['id'])
        print("Track name: " + tracks['tracks']['items'][i]['name'])
        print("Artist: " + tracks['tracks']['items'][i]['artists'][0]['name'])
        print("Track URL: " + tracks['tracks']['items'][i]['external_urls']['spotify'])
        print("Image URL: " + tracks['tracks']['items'][i]['album']['images'][0]['url'])
        print("\n")

Track id: 5vdp5UmvTsnMEMESIF2Ym7
Track name: Another One Bites The Dust - Remastered 2011
Artist: Queen
Track URL: https://open.spotify.com/track/5vdp5UmvTsnMEMESIF2Ym7
Image URL: https://i.scdn.co/image/ab67616d0000b273056e90910cbaf5c5b892aeba


Track id: 3JvKfv6T31zO0ini8iNItO
Track name: Another Love
Artist: Tom Odell
Track URL: https://open.spotify.com/track/3JvKfv6T31zO0ini8iNItO
Image URL: https://i.scdn.co/image/ab67616d0000b2731917a0f3f4152622a040913f


Track id: 4Vwg5CDYqAOf50Rt1WC6Kh
Track name: Just Another Interlude
Artist: Bryson Tiller
Track URL: https://open.spotify.com/track/4Vwg5CDYqAOf50Rt1WC6Kh
Image URL: https://i.scdn.co/image/ab67616d0000b273f62cefefc39ba2cb384889f5


Track id: 5pDnUwpCwOXvGvcjRkqvS4
Track name: Pour Me Another One
Artist: Krept & Konan
Track URL: https://open.spotify.com/track/5pDnUwpCwOXvGvcjRkqvS4
Image URL: https://i.scdn.co/image/ab67616d0000b2731f677ac696a536a69f57bad5


Track id: 5dPj0gss1usGcubIR2qCu5
Track name: Just Another Interlude
Art

In [ ]:
# Make Track Analysis Request for BPM etc.